In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

# 5. 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        # 2. 문서를 쪼개서 임베딩, Vector Store에 저장된 조각 문서를 retriever로 가져옴
        "context": retriver,
        # 3. question은 러너블패스스루를 이용해 그대로 전달
        "question": RunnablePassthrough(),
    }
    # 4. 프롬프트에 context와 question을 전달
    | prompt
    # 6. ChatOpenAI llm 호출
    | llm
)

# 1. 체인에 전달
chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building where Winston Smith resides. It has glass doors that let in gritty dust, a hallway that smells of boiled cabbage and old rag mats, and a poster of an enormous face with the caption "BIG BROTHER IS WATCHING YOU." The flat Winston lives in is seven flights up, and the building has a faulty lift due to the economy drive in preparation for Hate Week. The building is located in London, the chief city of Airstrip One, a province of Oceania.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 2054, 'total_tokens': 2160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-206ecb0b-1b59-470c-a752-1b8c07dbfe62-0')